In [16]:
!pip install pywin32
!pip install pandas
!pip install openpyxl


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import win32com.client as win32
from openpyxl import load_workbook

# Load the Excel file
#df = pd.read_excel('demo.xlsx')

# Create an instance of the Outlook application
outlook = win32.Dispatch('outlook.application')

signature_html = """
<div style="font-family: 'Segoe UI', sans-serif;">
    <p style="color: #4CAF50; font-size: 24px;">Badr Adnani<br>
    <span style="color: black; font-size: 12px;"><strong>Bachelor of Computer Science, Final Year</strong><br>
    <a href="mailto:bh86jl@student.sunderland.ac.uk" style="color: black; font-size: 12px;">bh86jl@student.sunderland.ac.uk</a><br>
    <strong>Create Britain</strong><br></span></p>
    <a href="https://www.createbritain.com/Profiles/badradnani"><img src="https://cdn-ukwest.onetrust.com/logos/6b0858fa-ec2c-4078-8a59-de179d25c13d/58e2182f-9bb3-47f6-8351-993f66f40c06/91d0b9df-c276-4bf1-baf3-07bbba727d34/UoS_Logo_BLACK.png" width="200" height="100" alt="Logo 1"></a><br><br>
    <a href="https://www.createbritain.com/"><img src="https://music.createbritain.com/images/logos/CreateBritain_Double.png" alt="Logo 2" width="200" height="100"></a>
</div>
"""




# Template for the email body
email_body_template = """
Dear {company_name},<br><br>

My name is Badr Adnani, a final year Bachelor of Computer Science student at the University of Sunderland. I'm excited to invite your organization to join <strong>Create Sunderland</strong>, a new branch of <strong>Create Britain</strong>, aimed at fostering community collaboration. This initiative is a partnership with the University of Sunderland and is pivotal to my final year project and professional development.<br><br>

Joining is <strong>free</strong>, and so are our services. I will personally ensure your organization is presented at its best, using information from your website to create a compelling profile. Upon your approval, we’ll make this profile public to enhance your visibility within the Sunderland community.<br><br>

This opportunity is not only crucial for my career but also offers significant exposure for your organization. I look forward to possibly collaborating with you and would be honored to feature your organization on our platform.<br><br>

Thank you for considering this invitation. I look forward to the possibility of working together to bring this project to life.<br><br>

Warmest regards,
"""

print("Emails prepared. Review before sending.")

Emails prepared. Review before sending.


In [3]:
import pandas as pd
import win32com.client as win32
from openpyxl import load_workbook


# Function to check if the email is valid
def is_valid_email(email):
    if pd.isnull(email) or not isinstance(email, str):
        return False
    return "@" in email and "." in email

# Function to update the status in the Excel sheet
def update_status_in_excel(file_path, index, status):
    workbook = load_workbook(filename=file_path)
    sheet = workbook.active
    cell = f'F{index+2}'  # Adjust if your status column is different
    sheet[cell] = status
    workbook.save(filename=file_path)
    workbook.close()

# Function to display the menu and get the choice
def display_menu():
    print("\nMenu:")
    print("1. Display email")
    print("2. Send email")
    print("3. Exit")
    choice = input("Enter your choice (1/2/3): ")
    return choice

# Load the Excel file
file_path = 'test.xlsx'  # Adjust the path to your actual Excel file
df = pd.read_excel(file_path)

# Create an instance of the Outlook application
outlook = win32.Dispatch('outlook.application')

# Initialize 'choice' by calling 'display_menu()' before entering the loop
choice = display_menu()

while choice != '3':
    if choice == '1':
        for index, row in df.iterrows():
            company_name = row['Company Name']
            email_address = row['Email']
            
            # If the company name is not null and the email is valid, display the email
            if not pd.isnull(company_name) and is_valid_email(email_address):
                print(f"Displaying email for: {company_name}")
                mail = outlook.CreateItem(0)
                mail.Subject = "Invitation to Join Create Sunderland"
                mail.HTMLBody = email_body_template.format(company_name=company_name) + signature_html
                mail.Display(True)  # This properly associates the display command with the mail object
                break  # Exit after displaying the first valid email
        else:
            # The 'else' block for a 'for' loop executes when the loop is not terminated by a 'break' statement
            print("No valid entry found for display.")
    elif choice == '2':
        emails_processed = False
        end_of_data_reached = False

        for index, row in df.iterrows():
            company_name = row['Company Name']
            email_address = row['Email']
            status = row.get('Status', '')

            # Check for completely empty row (assuming 'Company Name' is always required)
            if pd.isnull(company_name):
                if not emails_processed:  # Check if this is the first empty row after actual data
                    end_of_data_reached = True
                break  # Break the loop if the company name is empty

            if status == 'Sent':
                print(f"Email already sent to {company_name}.")
                continue

            if not is_valid_email(email_address):
                print(f"Invalid email for {company_name}. Skipping.")
                continue

            print(f"Ready to send email to {company_name}. Press 'Y' to confirm.")
            confirmation = input().lower()
            if confirmation == 'y':
                mail = outlook.CreateItem(0)
                mail.To = email_address
                mail.Subject = "Invitation to Join Create Sunderland"
                mail.HTMLBody = email_body_template.format(company_name=company_name) + signature_html
                # Commented for safety.
                #mail.Send()
                update_status_in_excel(file_path, index, 'Sent')
                print(f"Email sent to {company_name}. Status updated in Excel.")
                emails_processed = True
            else:
                print("Email sending aborted by user.")

        if not emails_processed and not end_of_data_reached:
            print("No emails were sent. Please check the data for valid entries.")
        elif end_of_data_reached:
            print("End of data reached. No more emails to send.")
        elif emails_processed:
            print("Some emails were sent successfully.")
        else:
            print("An unexpected error occurred while sending emails.")
        
    else:
        print("Invalid choice. Please choose again.")
    
    # Re-prompt the menu after handling the previous choice
    choice = display_menu()

print("Exiting the program.")  # Update 'choice' at the end of the loop to reflect the next menu selection



Menu:
1. Display email
2. Send email
3. Exit
Displaying email for: Badr Adnani

Menu:
1. Display email
2. Send email
3. Exit
Email already sent to Badr Adnani.
Email already sent to zouhair.
Invalid email for lts. Skipping.
End of data reached. No more emails to send.

Menu:
1. Display email
2. Send email
3. Exit
Email already sent to Badr Adnani.
Email already sent to zouhair.
Invalid email for lts. Skipping.
End of data reached. No more emails to send.

Menu:
1. Display email
2. Send email
3. Exit
Exiting the program.
